<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

1  Цели соревнования

На основе данных, предоставленных устроителями соревнования, разработать и выбрать лучшие модели машинного обучения. На основе моделей машинного обучения разработать решение, которое позволит предсказать закрытие стартапа.

2  Ход Исследования
1. Базу данных по стартапам я соберу из двух источников: файл 'kaggle_startups_train_27042024.csv' с обучающими данными по стартапам и файл 'kaggle_startups_test_27042024.csv' с данными по стартапам, закрытие которых необходимо предсказать. 
2. С данными я не знаком. Поэтому мне понадобится обзор данных. 
3. Я проверю таблицы на пропуски и дублирование, внесу исправления, не влияющие на общую целостность и качество анализа, добавлю необходимые столбцы и проведу исследовательский и корреляционный анализ. 
4. Для построения модели я использую технологию пайплайна. 
5. Для иследования важности входных признаков я использую SHAP-анализ. 

3  Этапы исследования:

3.1. Загрузка и ознакомление с данными,
3.2. Педварительная обработка,
3.3. Исследовательский анализ данных,
3.4. Разработка новых синтетических признаков,
3.5. Проверка на мультиколлинеарность,
3.6. Отбор финального набора обучающих признаков,
3.7. Выбор и обучение моделей,
3.8. Получение результат,
3.9. Анализ важности ее признаков,

4 Дополнительные задания:

4.1. Реализовать решение с использованием технологии pipeline (из библиотеки sklearn, imblearn),
4.2. Выполнить полноценный исследовательский анализ и сформулировать рекомендации позволяющие повысить шанс на успех стартапа.
4.3. Подготовить полноценный отчет по исследовательской работе Дата Сайнтиста.

In [ ]:
# Установим актуальную версию модуля sklearn
!pip install --upgrade scikit-learn==1.4.1.post1

In [ ]:
# импорт библиотек
import os
import math
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats as st
from matplotlib import pyplot as plt

!pip install phik -q
import phik                                                     # инструмент корреляционного анализ

!pip install shap -q
import shap                                                     # обеспечивает SHAP-анализ важности признаков

!pip install -U imbalanced-learn -quit  
from imblearn.over_sampling import SMOTE                        # инструмент сэмплирования данных

from scipy.stats import shapiro                                 # проверка гауссовского распределенеия
from scipy.stats import normaltest                              # проверка гауссовского распределенеия

from sklearn.model_selection import train_test_split            # селектор тренировочной и тестовой выборок
from sklearn.impute import SimpleImputer                        # класс для работы с пропусками
from sklearn.preprocessing import (                             # классы для преобразования данных
    QuantileTransformer,
    PowerTransformer,
    OneHotEncoder, 
    OrdinalEncoder,
    LabelEncoder,
    StandardScaler,
    MinMaxScaler,
    RobustScaler,
    PolynomialFeatures
)

from sklearn.preprocessing import QuantileTransformer

from sklearn.ensemble import RandomForestClassifier             # модель «Рандомный лес»
from sklearn.neighbors import KNeighborsClassifier              # модель kNN
from sklearn.linear_model import LogisticRegression             # модель логистической регрессии

from sklearn.pipeline import Pipeline                           # обеспечивает работу с пайплайнами
from sklearn.compose import ColumnTransformer                   # помогает работать с данными разного типа в одном наборе
from sklearn.model_selection import GridSearchCV                # инструмент для автоподбора гиперпараметров
from sklearn.model_selection import RandomizedSearchCV          # инструмент для автоподбора гиперпараметров

from sklearn.metrics import f1_score                            # метрики
from sklearn.metrics import make_scorer                         # инструмент создания метрик

pd.options.display.float_format = '{:.2f}'.format               # настройка формата вывода чисел

RANDOM_STATE = 42
TEST_SIZE = 0.25


In [ ]:
# Функция отмечает значением True строки с проблемными (дубликатными) названиями стартапов
def names_function(x):
    if type(x) != type(''):        
        return False
    for bname in names:
        if bname == x.lower():
            return True
    return False 

In [ ]:
# Функция проверки гауссовского распределения 
def check_stats(x):
    stat, p = shapiro(x)
    print('Распределение', x.name, 'shapiro p =', round(p, 3), '-', 'скорее гауссовское' if p > 0.05 else 'скорее не гауссовское')
    stat, p = normaltest(x)
    print('Распределение', x.name, 'normaltest p =', round(p,  3), '-', 'скорее гауссовское' if p > 0.05 else 'скорее не гауссовское')
    print()

In [ ]:
# Функция отрисовки диаграммы типа 'histplot' и диаграммы размаха. Аргументы: датасет, наименование столбца.
def plot_function_col(df, column):    
    plt.figure(figsize=[14, 5])
    plt.subplot(2, 1, 1) 
    sns.set(rc={"figure.figsize":(18, 5)})
    sns.histplot(data=df, x=column, bins=5000).set(title='Гистограмма и диаграмма размаха по признаку ' + column)    
    plt.subplot(2, 1, 2)    
    sns.boxplot(x=df[column]).set()  
    plt.show() 

In [ ]:
# Функция отрисовки диаграммы типа 'bar'. Аргументы: набор данных, заголовок, текст ylabel
def bar_function(seria, title, ylabel):
    feature = seria
    h_feature = feature.plot(kind='bar', figsize=(10, 5), grid=True)
    h_feature.set_title(title)
    h_feature.set_xlabel('Категории')
    h_feature.set_ylabel(ylabel)
    plt.show()    

Шаг 1. Загрузка и ознакомление с данными

In [ ]:
# Загрузим обучающую базу данных
df_1 = pd.read_csv('kaggle_startups_train_27042024.csv')

In [ ]:
# Выведем на экран
df_1.head(3)

In [ ]:
# посмотрим информацию
df_1.info()

In [ ]:
# Построим гистограммы численных столбцов для предварительного знакомства с характером данных
df_1.hist(bins=45, figsize=(10, 3))
plt.show()

In [ ]:
# Отследим столбцы с пропусками
for column in df_1.columns:    
    print(f'Столбец "{column}" имеет {52514 - df_1[column].count()} пропусков')

Пояснения к обучающей базе данных:

1. Таблица имеет 52514 записей, размещенных в 13 столбцах.
3. На гистограмме заметно, что столбец "funding_total_usd" имеет сильный выброс
2. Столбец "name" имеет 1 пропусков
3. Столбец "category_list" имеет 2465 пропусков
4. Столбец "funding_total_usd" имеет 10069 пропусков
5. Столбец "country_code" имеет 5501 пропусков
6. Столбец "state_code" имеет 6762 пропусков
7. Столбец "region" имеет 6358 пропусков
8. Столбец "city" имеет 6358 пропусков
9. Столбец "closed_at" имеет 47599 пропусков

In [ ]:
# Загрузим базу данных для предсказаний
df_2 = pd.read_csv('kaggle_startups_test_27042024.csv')

In [ ]:
# Выведем на экран
df_2.head(3)

In [ ]:
# посмотрим информацию
df_2.info()

In [ ]:
# Построим гистограммы численных столбцов для предварительного знакомства с характером данных
df_2.hist(bins=45, figsize=(10, 7))
plt.show()

In [ ]:
# Отследим столбцы с пропусками
for column in df_2.columns:    
    print(f'Столбец "{column}" имеет {13125 - df_2[column].count()} пропусков')

Пояснения к базе данных для предсказаний:

1. Таблица имеет 13125 записей, размещенных в 12 столбцах.
2. Столбец "category_list" имеет 591 пропусков
3. Столбец "funding_total_usd" имеет 2578 пропусков
4. Столбец "country_code" имеет 1382 пропусков
5. Столбец "state_code" имеет 1695 пропусков
6. Столбец "region" имеет 1589 пропусков
7. Столбец "city" имеет 1587 пропусков

Выводы по шагу "Загрузка и ознакомление с данными":

1. Таблицы-источники имеют неодинаковые наборы столбцов. 
2. Обучающая база содержит столбец с целевым признаком и дополнительный столбец 'closed_at' для явного указания стартапов, закрытых до 2018-01-01.
3. База данных для предсказаний содержит столбец 'lifetime'. отсутствующий в обучающей базе.
4. После предварительной обработки таблиц необходимо привести в соответствие наборы их столбцов.

Шаг 2. Педварительная обработка.

In [ ]:
# Пропуски в столбцах 'name', 'category_list', 'country_code', 'state_code', 'region', 'city' заменим зачением 'Unknown'
columns = ['name', 'category_list', 'country_code', 'state_code', 'region', 'city']
for col in columns:
    df_1[col].fillna('Unknown', inplace = True)
    df_2[col].fillna('Unknown', inplace = True)

In [ ]:
# Пропуски в столбце 'closed_at' df_1_1 заменим зачением '2018-01-01'
const_date = pd.to_datetime('2018-01-01')
df_1['closed_at'].fillna('2018-01-01', inplace = True)

In [ ]:
# В df_1 и df_2 переведем в формат даты столбцы 'founded_at', 'first_funding_at', 'last_funding_at', 'closed_at'
df_1['founded_at'] = pd.to_datetime(df_1['founded_at'])
df_1['first_funding_at'] = pd.to_datetime(df_1['first_funding_at'])
df_1['last_funding_at'] = pd.to_datetime(df_1['last_funding_at'])
df_1['closed_at'] = pd.to_datetime(df_1['closed_at'])
df_2['founded_at'] = pd.to_datetime(df_2['founded_at'])
df_2['first_funding_at'] = pd.to_datetime(df_2['first_funding_at'])
df_2['last_funding_at'] = pd.to_datetime(df_2['last_funding_at'])

In [ ]:
# Далее работаем с копиями таблиц-источников
df_1_1 = df_1.copy(deep=True)
df_2_1 = df_2.copy(deep=True)

In [ ]:
# Проверим явные дубликаты в таблице df_1_1
print('Явных дубликатов строк:', df_1_1.duplicated().sum())
# Проверим явные дубликаты в таблице df_2_1
print('Явных дубликатов строк:', df_2_1.duplicated().sum())

In [ ]:
# B таблице df_1_1 проверим дубликаты по столбцу 'name'
print('Явных дубликатов по столбцу name:', df_1_1.duplicated().sum())
df_1_1['name'] = df_1_1['name'].str.lower()
print('Подозрений на неявные дубликаты по столбцу name в df_1_1:', df_1_1['name'].duplicated().sum())
# B таблице df_2_1 проверим дубликаты по столбцу 'name'
print('Явных дубликатов по столбцу name:', df_2_1.duplicated().sum())
df_2_1['name'] = df_2_1['name'].str.lower()
print('Подозрений на неявные дубликаты по столбцу name в df_2_1:', df_2_1['name'].duplicated().sum())

In [ ]:
# Выявим неявные дубликаты по столбцу 'name'
value_counts_df_1_1 = df_1_1['name'].value_counts()
value_counts_df_2_1 = df_2_1['name'].value_counts()
# Преобразуем value_counts_df_1_1 в df и присвоим имена колонкам
df_value_counts_df_1_1 = pd.DataFrame(value_counts_df_1_1)
df_value_counts_df_1_1 = df_value_counts_df_1_1.reset_index()
df_value_counts_df_1_1.columns = ['unique_values', 'counts']
print(df_value_counts_df_1_1)
# Преобразуем value_counts_df_2_1 в df и присвоим имена колонкам
df_value_counts_df_2_1 = pd.DataFrame(value_counts_df_2_1)
df_value_counts_df_2_1 = df_value_counts_df_2_1.reset_index()
df_value_counts_df_2_1.columns = ['unique_values', 'counts']
print(df_value_counts_df_2_1)

In [ ]:
# Добавим в df_1_1 столбец для True-обозначения неявных дубликатов по столбцу 'name' и посчитаем их количество
names = df_value_counts_df_1_1['unique_values'].head(43)
df_1_1['dupl_name'] = df_1_1['name'].apply(names_function)
df_1_1['dupl_name'].value_counts()

In [ ]:
# Выведем на экран записи df_1_1 с неявными дубликатами по столбцу 'name'
df_1_1d = df_1_1[df_1_1['dupl_name'] == True]
print('Подозрение на неявняе дубликаты в df_1_1:')
print(df_1_1d.sort_values('name').head(4))
# Выведем на экран записи df_2_1 с неявными дубликатами в столбце 'name'
print()
print('Подозрение на неявняе дубликаты в df_2_1:')
selected = df_2_1.loc[(df_2_1['name'] == 'quip') | (df_2_1['name'] == 'spoke')].sort_values('name')
print(selected)

In [ ]:
# Удалим единственный неявный дубликат в базе df_1_1
df_1_1 = df_1_1.drop(index=3250)
df_1_1 = df_1_1.reset_index(drop=True)
df_1_1 = df_1_1.drop('dupl_name', axis=1)

In [ ]:
# Проверим изменения в таблицах
df_1_1.info()
print()
df_2_1.info()

Пояснения по столбцу 'name' обучающей базы данных:

1. Явных дубликатов по столбцу name: 0
2. Подозрений на неявные дубликаты по столбцу name: 43
3. Визуальная проверка строк с дублированными названиями стартапов показал, что неявным дупликатом является только строка под индексом 3250, так как там полностью совпадают дата создания и местоположение стартапа. Строка удалена.
4. В остальных строках эти значения не совпадают - то есть это не дубликаты.

Пояснения по столбцу 'name' базы данных для предсказаний:

1. Явных дубликатов по столбцу name: 0
2. Подозрений на неявные дубликаты по столбцу name: 2
3. Визуальная проверка строк с дублированными названиями стартапов показал, что значения столбцов в строках не совпадают, то есть неявные дубликаты остутствуют.

In [ ]:
# Пропуски в столбце 'funding_total_usd' заменим медианным зачением
df_1_1['funding_total_usd'].fillna(df_1_1['funding_total_usd'].median(), inplace = True)
df_2_1['funding_total_usd'].fillna(df_2_1['funding_total_usd'].median(), inplace = True)

In [ ]:
# Проверим на выбросы и нормальность числовые столбцы таблицы df_1_1
plot_function_col(df_1_1, 'funding_total_usd')
check_stats(df_1_1['funding_total_usd'])
plot_function_col(df_1_1, 'funding_rounds')
check_stats(df_1_1['funding_rounds'])

In [ ]:
# Посмотрим статистические сводки по столбцам 'funding_total_usd' и 'funding_rounds' в df_1_1
df_1_1.describe()

In [ ]:
# Избавимся от выбросов в столбце 'funding_total_usd'. Разумным представляется предел финансирования 200 млн. USD 
df_1_1 = df_1_1.loc[df_1_1['funding_total_usd'] < 200000000]
plot_function_col(df_1_1, 'funding_total_usd')
check_stats(df_1_1['funding_total_usd'])

In [ ]:
# Добавим в df_1_1 столбец 'lifetime' - время жизни стартапа
df_1_1['lifetime'] = (df_1_1['closed_at'] - df_1_1['founded_at']).dt.days

In [ ]:
# Добавим в df_1_1 столбец 'post_days' - количество дней с закрытия стартапа
df_1_1['post_days'] = (const_date - df_1_1['closed_at']).dt.days
# Добавим в df_2_1 столбец 'post_days' - количество дней с закрытия стартапа
df_2_1['post_days'] = (const_date - df_2_1['founded_at']).dt.days - df_2_1['lifetime']

Выводы по шагу 'Педварительная обработка'

1. В категориальных столбцах обеих таблиц пропуски заменены значением 'Unknown'. Это сделано чтобы в обучении участвовало как можно больше записей.
2. В солбце 'funding_total_usd' пропуски заменены медианными значениями.
3. В df_1_1 удалены записи с выбросами по солбцу 'funding_total_usd'. Предел финансирования выбран в 200 млн. USD.
4. Числовые значения в столбце 'funding_rounds' по сути являются категориальными значениями. Там аномальных значений не обнаружено.
5. После предварительной обработки таблица df_1_1 содержит 52016 записей, таблица df_2_1 содержит 13125 записей.
6. Для соответствия с df_2_1 в таблицу df_1_1 добавлен столбец 'lifetime'.
7. В обе таблицы добавлен столбец 'post_days' - количество дней с закрытия стартапа.

Шаг 3. Исследовательский анализ данных.

In [ ]:
# Сбросим индексы в df_1_1
df_1_1 = df_1_1.reset_index(drop=True)

In [ ]:
# Проверим изменения в таблице
df_1_1.info()

In [ ]:
# Значения по столбцу 'name' являются идентификаторами стартапов и не влияют на целевой признак.

In [ ]:
# Данные по столбцу 'category_list' содержат очень много категорий стартапов, описаны разнообразно и нестандартизованно.
# Для исследования их необходимо свести в несколько крупных категорий по прикладным областям: искусство, политика, 
# торговля, производство, услуги, транспорт, медицина, образование, СМИ, общепит, айти-технологии.

In [ ]:
# Исследуем процент закрытых стартапов в логарифмической зависимости от величины финансирования
df_1_1['funding_total_log'] = np.log2(df_1_1['funding_total_usd'])
df_2_1['funding_total_log'] = np.log2(df_2_1['funding_total_usd'])
df_1_1['funding_total_log'].describe()
 

In [ ]:
# Функция разделяет стартапы по столбцу 'funding_total_log'
def funding_log(x):
    n = 30
    closed_counts = [0] * n
    operating_counts = [0] * n
    closed_percent = [0] * n
    for i in range(0, 52016):
        for j in range(1, n):
            if x.loc[i,'funding_total_log'] < j:            
                if x.loc[i,'status'] == 'closed':
                    closed_counts[j] += 1
                if x.loc[i,'status'] == 'operating':
                    operating_counts[j] += 1
                break
    for p in range(0, n):
        try:
            closed_percent[p] = round(closed_counts[p] / (closed_counts[p] + operating_counts[p]) * 100, 2)
        except:
            closed_percent[p] = 0 
    return operating_counts, closed_counts, closed_percent

In [ ]:
# Распределим стартапы по бинсам столбца 'funding_total_log'
result_counts = funding_log(df_1_1)

In [ ]:
# Посмотрим диаграмму распределения действующих и закрытых стартапов по bins столбца 'funding_total_log'
y = pd.Series(result_counts[0]) 
bar_function(y, 'Действующие стартапы по категориям финансирования', 'Количество')
y = pd.Series(result_counts[1]) 
bar_function(y, 'Закрытые стартапы по категориям финансирования', 'Количество')
y = pd.Series(result_counts[2]) 
bar_function(y,  'Процент закрытых стартапов по категориям финансирования', 'Проценты')

In [ ]:
df_1_1['funding_total_log'].describe()

Пояснения:

1. Чтобы учесть логарифмическое распределение сумм финансирования, данные по столбцу обращены в логарифмы и распределны по линейным категориям в 30 бинсах.
2. Гистограмма распределения долей закрытых стартапов показывает, что закрытые статапы в наибольшей доле финансировались на суммы менее 100000 долларов.

In [ ]:
# Исследуем распределение стартапов по странам в df_1_1
country_counts = pd.DataFrame()
country_counts['count_all'] = df_1_1.groupby(['country_code'])['status'].count()

In [ ]:
# Соберем количество действующих и закрытых стартапов по странам
df_1_1_closed = df_1_1.query('status == "closed"')
df_1_1_operating = df_1_1.query('status == "operating"')
country_counts['count_closed'] =  df_1_1_closed.groupby('country_code')['status'].count()
country_counts['count_operating'] =  df_1_1_operating.groupby('country_code')['status'].count()
country_counts.fillna(0, inplace = True)
country_counts['ratio_closed'] = round(country_counts['count_closed'] / country_counts['count_all'] * 100, 2)
top_country_counts = country_counts.sort_values(by='ratio_closed', ascending=False).head(50)

In [ ]:
# Функция отрисовки диаграммы типа 'bar'. Аргументы: набор данных, заголовок, текст ylabel
def bar_function_2(y, title, ylabel):
    feature = y
    h_feature = feature.plot(kind='bar', figsize=(10, 3), grid=True)
    h_feature.set_title(title)
    h_feature.set_xlabel('Коды стран')
    h_feature.set_ylabel(ylabel)
    plt.show()   

In [ ]:
# Посмотрим диаграмму распределения стартапов по странам
country_counts = country_counts.head(50)
bar_function_2(top_country_counts['count_all'], 'Распределение стартапов по странам', 'Количество')
country_counts = country_counts.head(50)
bar_function_2(top_country_counts['count_closed'], 'Распределение закрытых стартапов по странам', 'Количество')
country_counts = country_counts.head(50)
bar_function_2(top_country_counts['ratio_closed'], 'Распределение процента закрытых стартапов по странам', 'Проценты')

Пояснения:

1. Распределение доли закрытых стартапов показывает, что за рассматриваемый период чаще всего стартапы закрывались в странах  третьего мира типа Сомали, Майотта, Грузии, Пуэрто-Рико и в России.

In [ ]:
# Исследуем распределения стартапов по количеству раундов финансирования в df_1_1
round_counts = pd.DataFrame()
round_counts['all'] = df_1_1.groupby(['funding_rounds'])['status'].count()

In [ ]:
# Соберем количество действующих и закрытых стартапов по количеству раундов финансирования
round_counts['closed'] =  df_1_1_closed.groupby('funding_rounds')['status'].count()
round_counts['operate'] =  df_1_1_operating.groupby('funding_rounds')['status'].count()
round_counts.fillna(0, inplace = True)
round_counts['ratio_closed'] = round(round_counts['closed'] / round_counts['all'] * 100, 2)
round_counts

In [ ]:
# Посмотрим диаграмму распределения стартапов по количеству раундов финансирования
bar_function_2(round_counts['all'], 'Распределение стартапов по раундам финансирования', 'Количество')
bar_function_2(round_counts['closed'], 'Распределение стартапов по раундам финансирования', 'Количество')
bar_function_2(round_counts['ratio_closed'], 'Распределение процента закрытых стартапов по раундам финансирования', 'Проценты')

Пояснения:

1. Распределение доли закрытых стартапов показывает, что за рассматриваемый период закрытые стартапы чаще всего имели не более одного раунда финансирования.

In [ ]:
# В df_1_1 и df_2_1 место дат финансирования проставим промежутки в днях
df_1_1['first_funding_gap'] = abs(df_1_1['first_funding_at'] - df_1_1['founded_at']).dt.days
df_1_1['last_funding_gap'] = abs(df_1_1['last_funding_at'] - df_1_1['first_funding_at']).dt.days
df_2_1['first_funding_gap'] = abs(df_2_1['first_funding_at'] - df_2_1['founded_at']).dt.days
df_2_1['last_funding_gap'] = abs(df_2_1['last_funding_at'] - df_2_1['first_funding_at']).dt.days

In [ ]:
# Проверим изменения в таблицах
df_1_1.info()
print()
df_2_1.info()

In [ ]:
# Посмотрим информацию по столбцу 'lifetime'
df_1_1['lifetime'].describe()

In [ ]:
# Функция распределяет стартапы на категории по столбцу 'lifetime'
def lifetime(x):
    n = 30
    closed_counts = [0] * n
    operating_counts = [0] * n
    closed_percent = [0] * n
    for i in range(0, 52016):
        for j in range(1, n):
            if x.loc[i,'lifetime'] < j * 600:            
                if x.loc[i,'status'] == 'closed':
                    closed_counts[j] += 1
                if x.loc[i,'status'] == 'operating':
                    operating_counts[j] += 1
                break
    for p in range(0, n):
        try:
            closed_percent[p] = round(closed_counts[p] / (closed_counts[p] + operating_counts[p]) * 100, 2)
        except:
            closed_percent[p] = 0 
    return operating_counts, closed_counts, closed_percent

In [ ]:
# Распределим стартапы на категории по бинсам столбца 'lifetime'
result_lifetime = lifetime(df_1_1)

In [ ]:
# Посмотрим диаграмму распределения действующих стартапов по bins столбца 'lifetime'
y = pd.Series(result_lifetime[0]) 
bar_function(y, 'Действующие стартапы по bins столбца lifetime', 'Количество')

# Посмотрим диаграмму распределения действующих стартапов по bins столбца 'lifetime'
y = pd.Series(result_lifetime[1]) 
bar_function(y, 'Закрытые стартапы по bins столбца lifetime', 'Количество')

# Посмотрим диаграмму распределения действующих стартапов по bins столбца 'lifetime'
y = pd.Series(result_lifetime[2]) 
bar_function(y, 'Процент закрытых стартапов по bins столбца lifetime', 'Процент')

Пояснения:

1. Распределение доли закрытых стартапов показывает, что за рассматриваемый период закрытые стартапы чаще всего имели время жизни не более трех лет.

Выводы по шагу "Исследовательский анализ данных":

1. Чтобы учесть логарифмическое характер распределения сумм финансирования стартапов, данные по столбцу обращены в логарифмы и распределены по линейным категориям в 30 бинсах.
2. Построены диаграммы распределения действующих и закрытых стартапов, а также диаграммы распределения долей закрытых стартапов в зависимости от различных факторов. Они показывают, что:
3. Закрытые статапы в наибольшей доле финансировались на суммы менее 100000 долларов.
4. За рассматриваемый период чаще всего стартапы закрывались в странах третьего мира типа Сомали, Майотта, Грузии, Пуэрто-Рико и в России.
5. Закрытые стартапы чаще всего имели не более одного раунда финансирования.
5. Закрытые стартапы чаще всего имели время жизни не более трех лет.

Шаг 4. Разработка новых синтетических признаков.

In [ ]:
df_1_1.head(3)

Новый признак 'category_funding_log'

In [ ]:
# Посмотрим описания столбца 'funding_total_log' в df_1_1 и df_2_1
print('funding_total_log df_1_1:')
print(df_1_1['funding_total_log'].describe())
print()
print('funding_total_log df_2_1:')
print(df_2_1['funding_total_log'].describe())

In [ ]:
# Функция распределяет стартапы на категории 'category_funding_log' (30)
def funding_category_func(x):
    for i in range(1, 3):
        if x > i * 10 - 10 and x < i * 10:            
            return i
    return 3

In [ ]:
# Закодируем стартапы в df_1_1 и df_2_1 по категориям 'funding_total_log'
df_1_1['category_funding_log'] = df_1_1['funding_total_log'].apply(funding_category_func)
df_2_1['category_funding_log'] = df_2_1['funding_total_log'].apply(funding_category_func)
df_1_1.head(3)

Новый признак 'category_lifetime'

In [ ]:
# Посмотрим описания столбца 'lifetime' в df_1_1 и df_2_1
print('lifetime df_1_1:')
print(df_1_1['lifetime'].describe())
print()
print('lifetime df_2_1:')
print(df_2_1['lifetime'].describe())

In [ ]:
# Функция перевода дней по столбцу 'lifetime' в категории (30)
def category_lifetime_func(x):
    for i in range(500, 17500, 500):
        if x < i:            
            return round(i / 500)
    return 35

In [ ]:
# Переведем дни по столбцу 'lifetime' в категории
df_1_1['category_lifetime'] = df_1_1['lifetime'].apply(category_lifetime_func)
df_2_1['category_lifetime'] = df_2_1['lifetime'].apply(category_lifetime_func)

Новый признак 'category_location'

In [ ]:
def location_func(x):
    if x == 'RUS':
        return 1
    elif x == 'USA':
        return 2
    else:
        return 3

In [ ]:
# Распределим стартапы по категориям стран локации
df_1_1['category_location'] = df_1_1['country_code'].apply(location_func)
df_2_1['category_location'] = df_2_1['country_code'].apply(location_func)

Новый признак 'category_firstgap'

In [ ]:
# Посмотрим описания столбца 'first_funding_gap' в df_1_1 и df_2_1
print('first_funding_gap df_1_1:')
print(df_1_1['first_funding_gap'].describe())
print()
print('first_funding_gap df_2_1:')
print(df_2_1['first_funding_gap'].describe())

In [ ]:
# Функция перевода дней по столбцу 'first_funding_gap' в категории (45)
def category_firstgap_func(x):
    for i in range(0, 45):
        if x > i * 365 and x < i * 365 + 365:            
            return i
    return 35

In [ ]:
# Переведем дни по столбцу df_1_1 'first_funding_gap' в категории
df_1_1['category_firstgap'] = df_1_1['first_funding_gap'].apply(category_firstgap_func)
df_2_1['category_firstgap'] = df_2_1['first_funding_gap'].apply(category_firstgap_func)
df_1_1.head(3)

Новый признак 'category_lastgap'

In [ ]:
# Посмотрим описания столбца 'last_funding_gap' в df_1_1 и df_2_1
print('last_funding_gap df_1_1:')
print(df_1_1['last_funding_gap'].describe())
print()
print('last_funding_gap df_2_1:')
print(df_2_1['last_funding_gap'].describe())

In [ ]:
# Функция перевода дней по столбцу 'last_funding_gap' в категории (30)
def category_lastgap_func(x):
    for i in range(0, 30):
        if x > i * 365 and x < i * 365 + 365:            
            return i
    return 35

In [ ]:
# Переведем дни по столбцу df_1_1 'last_funding_gap' в категории
df_1_1['category_lastgap'] = df_1_1['last_funding_gap'].apply(category_lastgap_func)
df_2_1['category_lastgap'] = df_2_1['last_funding_gap'].apply(category_lastgap_func)
df_1_1.head(3)

In [ ]:
df_1_1.info()

Выводы по шагу "Разработка новых синтетических признаков":

1. Для поиска наиболее оптимального набора признаков в таблицы добавлены новые столбцы.
2. Столбец 'category_funding_log'
3. Столбец 'category_lifetime'
4. Столбец 'category_location'
5. Столбец 'category_firstgap'
6. Столбец 'category_lastgap'

Шаг 5. Корреляционный анализ.

In [ ]:
# Имеем следующие признаки, которые можно комбинировать для получения лучшей модели:
columns = ['name', 'category_list', 'funding_total_usd', 'status', 'country_code', 'funding_rounds', 'founded_at', 'closed_at'] 
new_columns = ['lifetime', 'post_days', 'funding_total_log', 'first_funding_gap', 'last_funding_gap']
cat_columns = ['category_funding_log', 'category_lifetime', 'category_location', 'category_firstgap', 'category_lastgap']

In [ ]:
# Создадим финальную копию обучающего датасета
df_train = df_1_1[['lifetime', 'post_days', 'status']]
df_train = df_train.reset_index(drop=True)

In [ ]:
# Создадим финальную копию целевого датасета
df_test = df_2_1[['name', 'post_days', 'lifetime']]

In [ ]:
# В столбце 'status' df_train применим 1 и 0
df_train['status'] = df_train['status'].apply(lambda x: 0 if x == 'operating' else 1)

In [ ]:
# Выберем численные столбцы для анализа
num_columns = ['lifetime', 'post_days', 'status']

In [ ]:
# Вычислим матрицу корреляции с использованием phik
corr_matrix = df_train.phik_matrix(interval_cols=num_columns)
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Phi_K Correlation Matrix')
plt.show()

Выводы по шагу "Разработка новых синтетических признаков":

1. В поисках наиболее оптимального результата было составлено около 100 наборов входных признаков.
2. Самый лучший результат дал набор из признаков 'lifetime' и 'post_days'.
3. Мультикорреляция между этими признаками отсутствует.

Шаг 6 и 7. Выбор и обучение моделей. Получение результат.

In [ ]:
# Обучающий датасет готов
df_train.reset_index(drop=True)
df_train.head(3)

In [ ]:
df_train.info()

In [ ]:
# Тестовый датасет готов
#df_test.reset_index(drop=True)
df_test.head(3)

In [ ]:
df_test.info()

In [ ]:
# Соберем входные признаки по типам:
num_columns = ['lifetime', 'post_days']
ohe_columns = []
ord_columns = []

In [ ]:
# Создадим тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(
    df_train.drop(['status'], axis=1),
    df_train['status'],
    test_size = 0.25, 
    random_state = RANDOM_STATE,
    stratify = df_train['status']
)

In [ ]:
# Сбалансируем доли успешных и закрытых стартапов
sm = SMOTE(random_state=42, k_neighbors=5)
X_train_resample, y_train_resample = sm.fit_resample(X_train, y_train)
y_train_resample.value_counts()

In [ ]:
# Функция построения пайплайн на основе модели линейной регрессии. Возвращает y_test_pred, y_test, f1_test.
def pipeline_func(X_train, X_test, y_train, y_test):
    ohe_pipe = Pipeline(
        [
            (
                'ohe',                                                              
                OneHotEncoder(                                                      
                    drop=None, 
                    handle_unknown='ignore',               
                    categories='auto')        
            )
        ]
    )
    # Соберем пайплайн подготовки данных
    data_preprocessor = ColumnTransformer(
        [
            ('ohe', ohe_pipe, ohe_columns),
            ('num', MinMaxScaler(), num_columns)                                     
        ]     
    )
    # Соберем итоговый пайплайн
    pipe_final= Pipeline(
        [
            ('preprocessor', data_preprocessor),
            ('model', RandomForestClassifier(random_state=RANDOM_STATE))         
        ]
    )
    # Составим гиперпараметры для моделей
    param_distributions = [
        # словарь для модели KNeighborsClassifier() 
        {        
            'model': [KNeighborsClassifier()],
            # указываем гиперпараметр модели
            'model__n_neighbors': range(1, 20),
            # указываем список методов масштабирования
            'preprocessor__num': [StandardScaler(), MinMaxScaler()]   
        },
        # словарь для модели RandomForestClassifier()
        {
            'model': [RandomForestClassifier(random_state=RANDOM_STATE)],
            'model__max_features': range(2, 6),
            'model__max_depth': range(2, 12),
            'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
        },
        # словарь для модели LogisticRegression()
        {
            'model': [LogisticRegression(random_state=RANDOM_STATE, 
                solver='liblinear', 
                penalty='l1')],
            'model__C': range(1, 4),
            'preprocessor__num': [StandardScaler(), MinMaxScaler(), 'passthrough']  
        } 
    ]    
    # Настроим поиск по гиперпараметрам
    randomized_search = RandomizedSearchCV(
        pipe_final,
        param_distributions=param_distributions, 
        scoring='f1',
        random_state=RANDOM_STATE,
        n_iter=10,
        verbose=10,
        n_jobs=-1
    )
    # Обучим модель
    randomized_search.fit(X_train, y_train)
    # Выведем параметры лучшей модели
    print('Лучшая модель и её параметры:\n\n', randomized_search.best_estimator_) 
    print('Метрика лучшей модели на кросс-валидации:', randomized_search.best_score_)
    # Применим лучшую модель к тренировочной выборке
    y_train_pred = randomized_search.predict(X_train)
    print(f'Метрика F1 на тренировочной выборке: {round(f1_score(y_train, y_train_pred), 2)}')    
    # Применим лучшую модель к тестовой выборке тренировочного датасета
    y_test_pred = randomized_search.predict(X_test)
    score = round(f1_score(y_test, y_test_pred, pos_label=1), 2)
    print(f'Метрика F1 на тестовой выборке: {score}')
    y_test_predict_proba = randomized_search.predict_proba
    # Применим лучшую модель к тестовому датасету
    y_test_pred_1 = randomized_search.predict(df_test)
    
    return y_test_pred, y_test, score, y_test_pred_1, y_test_predict_proba

pred_result = pipeline_func(X_train_resample, X_test, y_train_resample, y_test)

In [ ]:
pred_result[3]

In [ ]:
# Преобразуем list в series
pred_series = pd.Series(pred_result[3])

In [ ]:
# Создадим итоговый датасет с названиями и предсказанными статусами стартапов
result = pd.DataFrame()
result['name'] = df_2['name']
result['status'] = pred_series
result['status'] = result['status'].apply(lambda x: 'operating' if x == 0 else 'closed')
result

In [ ]:
# Проконтролируем результаты предсказания
result['status'].value_counts()

In [ ]:
# Выгрузим в файл
result.to_csv('output_100.csv', index=False)

Выводы по шагу "Выбор и обучение моделей. Получение результат.":

1. Самой успешной оказалась модель RandomForestClassifier(max_depth=7, max_features=3, random_state=42).
2. Самый лучший результат метрики F1 = 1.00

Шаг 8. Проверка важность признаков.

In [ ]:
# Получим выборку из 1000 рандомных строк df_1
df_shap2 = df_train.sample(1000)
df_shap2.info()

In [ ]:
# Проверим распределение статусов
df_shap2['status'].value_counts()

In [ ]:
# Создадим тренировочную и тестовую shap-выборки
X_train_shap2, X_test_shap2, y_train_shap2, y_test_shap2 = train_test_split(
    df_shap2.drop(['status'], axis=1),
    df_shap2['status'],
    test_size = TEST_SIZE, 
    random_state = RANDOM_STATE,
    stratify = df_shap2['status']
)

In [ ]:
# Применим пайплайн!функцию к shap-выборкам
shap2_result = pipeline_func(X_train_shap2, X_test_shap2, y_train_shap2, y_test_shap2)

In [ ]:
# Проведем SHAP-анализ важности признаков
explainer = shap.KernelExplainer(shap2_result[4], X_test_shap2, keep_index=True)
shap_values2 = explainer(X_test_shap2)

In [ ]:
# Помотрим размеры таблицы результатов SHAP-анализа
shap_values2.shape

In [ ]:
# Для построения графика возьмем экземпляр с одним измерением
shap_values2 = shap_values2[:, :, 1]
shap_values2.shape

In [ ]:
# Визуализируем вклад каждого признака в классификацию всех наблюдений
shap.plots.beeswarm(shap_values2) 

In [ ]:
# Визуализируем индивидуальные SHAP-значения отдельного наблюдения
shap.plots.waterfall(shap_values2[5])  

In [ ]:
# Визуализируем общий вклад признаков в прогнозы модели
shap.plots.bar(shap_values2) 

Выводы по шагу "Проверка важность признаков":

1. Наибольший вклад в обученик и прогнозирование вносит признак 'post_days'.

Шаг 8. Выводы и рекомендации.

Выводы:

1. Были использованы два источника: обучающая база данных имеет 52514 записей и целевая тестовая база - 13125 записей. 
2. Источники имеют неодинаковые наборы столбцов. Обучающая база содержит столбец с целевым признаком и дополнительный столбец 'closed_at' для явного указания стартапов, закрытых до 2018-01-01. База данных для предсказаний содержит столбец 'lifetime'. отсутствующий в обучающей базе.
3. Источники практически не имут дубликатов, но имеют многочисленные пропуски в данных. Дубликаты удалены. Пропуски заполнены.
4. В категориальных столбцах обеих таблиц пропуски заменены значением 'Unknown'. Это сделано чтобы в обучении участвовало как можно больше записей.
5. В солбце 'funding_total_usd' пропуски заменены медианными значениями.
6. В df_1_1 удалены записи с выбросами по солбцу 'funding_total_usd'. Предел финансирования выбран в 200 млн. USD.
7. После предварительной обработки таблица df_1_1 содержит 52016 записей, таблица df_2_1 содержит 13125 записей.
8. Для соответствия с df_2_1 в таблицу df_1_1 добавлен столбец 'lifetime'.
9. В обе таблицы добавлен столбец 'post_days' - количество дней с закрытия стартапа.
10. Чтобы учесть логарифмическое характер распределения сумм финансирования стартапов, данные по столбцу 'funding_total_usd' обращены в логарифмы.
11. Построены диаграммы распределения действующих и закрытых стартапов, а также диаграммы распределения долей закрытых стартапов в зависимости от различных факторов.
12. Выяснилось, что закрытые статапы в наибольшей доле финансировались на суммы менее 100000 долларов.
13. За рассматриваемый период чаще всего стартапы закрывались в странах третьего мира типа Сомали, Майотта, Грузии, Пуэрто-Рико и в России.
14. Закрытые стартапы чаще всего имели не более одного раунда финансирования.
15. Закрытые стартапы чаще всего имели время жизни не более трех лет.
16. Введены новые входные признаки: 'category_funding_log' - логарифм от суммы финансирования стартапа, 'category_lifetime' - категория по времени жизни стартапа, 'category_location' - категория от страны-местоположения стартапа, 'category_firstgap' - категория от промежутка в днях между открытием стартапа и первым раундом финансирования, 'category_lastgap' - категория от промежутка в днях между первым  и последним раундоми финансирования.
17. В поисках наиболее оптимального результата для обучения модели МО было использовано около 100 наборов входных признаков.
18. Мультикорреляция между признаками отсутствовала.
19. Самый лучший результат дал набор из признаков 'lifetime' и 'post_days'.
20. Самой успешной оказалась модель RandomForestClassifier(max_depth=7, max_features=3, random_state=42).
21. Самый лучший результат имеет метрику F1 = 1.00
22. Каждый набор признаков исследовался на важность. 
23. В самом успешном случае наибольший вклад в обучение и прогнозирование внес признак 'post_days'.

Рекомендации:

1. Как показал исследовательский анализ данных, при прочих равных условиях, наиболее значительное влияние на успешность стартапа имеют систематические финасовые вложения на протяжении не менее трех лет. 
2. Общая сумма финансовых вложений желательно должна составлять не менее 100 тысяч долларов. 